In [ ]:
using DifferentialEquations
using PyPlot

In [ ]:
cd("../rms")
include("Parse.jl")
include("Reactor.jl")

In [ ]:
phaseDict = readinput("testing/superminimal.yml") #load mechanism dictionary

In [ ]:
spcs = phaseDict["gas"]["Species"]; #mechanism dictionaries index:  phaseDict[phasename]["Species" or "Reactions"]
rxns = phaseDict["gas"]["Reactions"];

In [ ]:
ig = IdealGas(spcs,rxns,name="gas") #Define the phase (how species thermodynamic and kinetic properties calculated)
initialconds = Dict(["T"=>1000.0,"P"=>10.0e5,"H2"=>0.67,"O2"=>0.33]) #Set simulation Initial Temp and Pressure
state = MolarState(initialconds,ig) #Define the initial state of the system
domain = ConstantVDomain(state=state,phase=ig) #Define the domain (encodes how system thermodynamic properties calculated)

In [ ]:
react = BatchSingleDomainReactor(domain,(0.0,0.101)); #Create the reactor object

In [ ]:
sol = solve(react.ode,CVODE_BDF(),abstol=1e-20,reltol=1e-12); #solve the ode associated with the reactor

In [ ]:
ts = exp.(range(log(1e-15),length=10000,stop=log(0.1)))
xs = [sol(t)[1:end-1]./sum(sol(t)[1:end-1]) for t in ts]
xmat = hcat(xs...)
maxes = [maximum(xmat[i,:]) for i = 1:size(xmat)[1]]
thresh = 1e-9
spnames = []
for i = 1:length(maxes)
    if maxes[i] > thresh
        plot(ts,xmat[i,:])
        push!(spnames,ig.species[i].name)
    end
end
legend(spnames)

In [ ]:
plot(ts,[sol(t)[end] for t in ts]) #Temperature vs time

In [ ]:
ts[argmax(diff([sol(t)[end] for t in ts]))] #Ignition Delay Time in seconds